In [55]:
from collections import Counter

class Hand:
    def __init__(self, cards, points) -> None:
        self.cards = cards
        self.points = points
        self.prev_hand = None
        self.next_hand = None

def get_raw_score(hands):
    raw_scores = dict()
    ranks = {
        5: 7,
        4: 6,
        (3, 2): 5,
        3: 4,
        (2, 2): 3,
        2: 2,
    }
    for h in hands:
        c = Counter(h.cards)
        j = 0
        if "J" in c:
            j = c["J"]
            del c["J"]
        r = list(c.values())
        r.sort()
        if (r[-1], r[-2]) in ranks:
            if ranks[(r[-1], r[-2])] not in raw_scores:
                raw_scores[ranks[(r[-1], r[-2])]] = list()
            raw_scores[ranks[(r[-1], r[-2])]].append(h)
        elif (r[-1]) in ranks:
            if ranks[r[-1]] not in raw_scores:
                raw_scores[ranks[r[-1]]] = list()
            raw_scores[ranks[r[-1]]].append(h)
        else:
            if 1 not in raw_scores:
                raw_scores[1] = list()
            raw_scores[1].append(h)

    return raw_scores

input = open('test').readlines()

hands = list()
for line in input:
    cards, score = line.strip().split(' ')
    hands.append(Hand(cards, int(score)))
d = get_raw_score(hands)
print(d)

{2: [<__main__.Hand object at 0x7f3da4b33f10>, <__main__.Hand object at 0x7f3d8ecf3eb0>], 4: [<__main__.Hand object at 0x7f3d8ecf2950>, <__main__.Hand object at 0x7f3d8ecf06d0>], 3: [<__main__.Hand object at 0x7f3d8ecf20e0>]}


In [32]:
base_hand = hands[0]
for h in hands[1:]:
    while(1):
        print(f"In:{h.points}\tCurr:{base_hand.points}")
        if h.points > base_hand.points:
            if base_hand.prev_hand is not None:
                if base_hand.prev_hand.points > h.points:
                    h.prev_hand = base_hand.prev_hand
                    h.next_hand = base_hand
                    base_hand.prev_hand.next_hand = h
                    base_hand.prev_hand = h
                else:
                    print("moving prev")
                    base_hand = base_hand.prev_hand
            else:
                base_hand.prev_hand = h
                h.next_hand = base_hand
        elif h.points < base_hand.points:
            if base_hand.next_hand is not None:
                if base_hand.next_hand.points < h.points:
                    h.next_hand = base_hand.next_hand
                    h.prev_hand = base_hand
                    base_hand.next_hand.prev_hand = h
                    base_hand.next_hand = h
                else:
                    print('moving next')
                    base_hand = base_hand.next_hand
            else:
                base_hand.next_hand = h
                h.prev_hand = base_hand
        else:
            print("hand in correct spot")
            break


In:684	Curr:765
moving next
In:684	Curr:684
hand in correct spot
In:28	Curr:684
moving next
In:28	Curr:483
moving next
In:28	Curr:220
moving next
In:28	Curr:28
hand in correct spot
In:220	Curr:28
moving prev
In:220	Curr:220
hand in correct spot
In:483	Curr:220
moving prev
In:483	Curr:483
hand in correct spot


In [44]:
d

{1: [<__main__.Hand at 0x7f3d8e7a3880>,
  <__main__.Hand at 0x7f3d8e7a3a00>]}